<a href="https://colab.research.google.com/github/Cloud-Course-Group-Phoenix/Project-Pheonix/blob/main/Logic/UserManager.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os, sys

try:
    # Clone the GitHub repository if not already present
    if not os.path.exists("/content/Project-Pheonix"):
        !git clone https://github.com/Cloud-Course-Group-Phoenix/Project-Pheonix.git /content/Project-Pheonix

    # Change directory to project root
    %cd /content/Project-Pheonix

    # Checkout the 'main' branch
    !git fetch origin -q
    !git checkout main -q

    # Add project directory to Python path
    sys.path.append("/content/Project-Pheonix/Logic")

    # Install required Python packages
    %pip install -q firebase
    %pip install -q importnb

    print("✅ Setup completed successfully.")

    # Imports
    from firebase import firebase
    from IPython.display import clear_output
    from importnb import Notebook
    with Notebook():
        import CloudDB as dbService

except Exception as e:
    print("❌ Setup failed:", str(e))

# Clear installation output for cleaner display
clear_output()

In [ ]:
def login(username, password):
    if not username or not password:
        return "❌ Please fill in all fields.", None, False

    user_data = dbService.get_user_by_username(username)

    # Check if user exists
    if not user_data:
        return "❌ Invalid username or password.", None, False

    # Verify password
    if user_data["password"] != password:
        return "❌ Invalid username or password.", None, False

    # Return success message, username, and admin status
    if user_data.get("is_admin"):
        return f"✅ Welcome Admin {username}!", username, user_data.get("is_admin")
    else:
        return f"✅ Welcome {username}!", username, user_data.get("is_admin")


def register_account(username, password, confirm_password, is_admin):
    if not username or not password:
        return "❌ Please fill in all fields.", None, False
    if password != confirm_password:
        return "❌ Passwords do not match.", None, False

    # Use Firebase's direct method to check if user exists
    existing_user = dbService.get_user_by_username(username)

    if existing_user is not None:
        return "❌ Username already exists.", None, False

    user_data = {
        "password": password,
        "is_admin": is_admin
        "coins": 2500
    }

    # Save user directly to Firebase
    result = dbService.insert_to_db_user(username, user_data)
    if result:
        return f"✅ User '{username}' registered successfully!", username, is_admin
    else:
        return "❌ Failed to register. Try again later.", None, False
